In [10]:

#Codigo nuevo
import numpy
import scipy
import numba
import matplotlib
import pandapower as pp
import pandapower.networks
import pandapower.topology
import pandapower.plotting
import pandapower.converter
import pandapower.estimation
#Crear red vacia (net)
net = pp.create_empty_network()
#Crear barras
b1 = pp.create_bus(net, vn_kv=20., name="Bus 1")
b2 = pp.create_bus(net, vn_kv=0.4, name="Bus 2")
pp.create_ext_grid(net, bus=b1, vm_pu=1, name="Conexion de malla");
pp.create_load(net, bus=b2, p_mw=0.999877, q_mvar=0.015707, name="Carga");
#Crear linea
pp.create_line_from_parameters(net, from_bus=b1, to_bus=b2, length_km=500, r_ohm_per_km=0.02, x_ohm_per_km=0.115, c_nf_per_km=19.1, max_i_ka=1)
pp.runpp(net)



In [11]:
import numpy as np
import cmath

# Parámetros de la línea
r = 0.02  # resistencia por km
x = 0.115  # reactancia por km
c = 19.1 * 1e-9  # capacitancia por km en faradios
l = 500  # longitud de la línea en km
f = 50  # frecuencia en Hz

# Convertir la capacitancia a la admitancia
omega = 2 * np.pi * f
y = 1j * omega * c

# Parámetros de la línea por unidad de longitud
Z = r + 1j * x
Y = y

# Parámetros de la línea total
Z_total = Z * l
Y_total = Y * l

# Cálculo de los parámetros ABCD para una línea larga
gamma = cmath.sqrt(Z_total * Y_total)
Zc = cmath.sqrt(Z_total / Y_total)

A = cmath.cosh(gamma)
B = Zc * cmath.sinh(gamma)
C = (1 / Zc) * cmath.sinh(gamma)
D = A

# Creación de la matriz ABCD
ABCD = np.array([[A, B], [C, D]])

print("Matriz ABCD:")
print(ABCD)

Matriz ABCD:
[[ 9.14939702e-01+1.45734594e-02j  9.43228339e+00+5.59096754e+01j]
 [-1.47449763e-05+2.91467707e-03j  9.14939702e-01+1.45734594e-02j]]


In [12]:
#Visualizar
net.res_bus #Visualizar resultados de barras


,vm_pu,va_degree,p_mw,q_mvar
0,1.000000,0.000000,-1.031924,1.065891
1,1.053387,-8.735714,0.999877,0.015707


In [13]:
net.res_line #Visualizar resultados de lineas

,p_from_mw,q_from_mvar,p_to_mw,q_to_mvar,pl_mw,ql_mvar,i_from_ka,i_to_ka,i_ka,vm_from_pu,va_from_degree,vm_to_pu,va_to_degree,loading_percent
0,1.031924,-1.065891,-0.999877,-0.015707,0.032047,-1.081598,0.042827,1.370224,1.370224,1.0,0.0,1.053387,-8.735714,137.02237
